In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import math
import yfinance as yf
from datetime import datetime, timedelta
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers import Dropout
from sklearn.preprocessing import MinMaxScaler


In [2]:
data=yf.download(tickers='EURUSD=X',start='2011-12-31',end='2019-12-31',interval ='1d')
df = df.filter(['Close'])
dataset = df.values
train_len = math.ceil( len(dataset) *.8) 
scaler = MinMaxScaler(feature_range = (0, 1))
training_data = scaler.fit_transform(dataset[:train_len , : ]) 
x_train =[] 
y_train =[] 

for i in range(60, len(training_data)):
    x_train.append(training_data[i-60:i,0])
    y_train.append(training_data[i,0])
    
x_train,y_train = np.array(x_train), np.array(y_train)
x_train= np.reshape(x_train,( x_train.shape[0] ,x_train.shape[1] ,1))

[*********************100%***********************]  1 of 1 completed


In [12]:
df=yf.download(tickers='EURUSD=X',start='2011-12-31',end='2019-12-31',interval ='1d')
df

[*********************100%***********************]  1 of 1 completed


,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
2011-12-30,1.295219,1.299596,1.290606,1.295605,1.295605,0
2012-01-02,1.296008,1.296395,1.292257,1.296008,1.296008,0
2012-01-03,1.293628,1.306592,1.293628,1.294096,1.294096,0
2012-01-04,1.305398,1.307395,1.290206,1.305602,1.305602,0
2012-01-05,1.293092,1.294297,1.278527,1.293142,1.293142,0
...,...,...,...,...,...,...
2019-12-24,1.109299,1.109755,1.107064,1.109385,1.109385,0
2019-12-25,1.109139,1.110864,1.107788,1.109016,1.109016,0
2019-12-26,1.109509,1.110494,1.108279,1.109545,1.109545,0


In [3]:
test_data = scaler.transform(dataset[train_len - 60: , : ])
x_test = []
y_test = []

for i in range(60,len(test_data)):
    x_test.append(test_data[i-60:i,0])
    y_test.append(test_data[i,0])
    
x_test, y_test = np.array(x_test), np.array(y_test)
x_test = np.reshape(x_test, (x_test.shape[0],x_test.shape[1],1))

In [4]:
forexPredictor = Sequential()
forexPredictor.add(LSTM(50, return_sequences = True, input_shape=(x_train.shape[1], 1)))
forexPredictor.add(Dropout(0.2))
forexPredictor.add(LSTM( 50, return_sequences = False))
forexPredictor.add(Dropout(0.2))
forexPredictor.add(Dense(50))
forexPredictor.add(Dense(1))
forexPredictor.compile(optimizer = 'adam', loss = 'mean_squared_error')
forexPredictor.save('forex_model.h5')

In [5]:
forexPredictor.fit(x_train, y_train, validation_data=(x_test,y_test), epochs = 100, batch_size = 32)

Epoch 1/100
51/51 [==============================] - 8s 63ms/step - loss: 0.0681 - val_loss: 4.4937e-04
Epoch 2/100
51/51 [==============================] - 2s 35ms/step - loss: 0.0049 - val_loss: 8.2310e-04
Epoch 3/100
51/51 [==============================] - 2s 35ms/step - loss: 0.0044 - val_loss: 4.2949e-04
Epoch 4/100
51/51 [==============================] - 2s 34ms/step - loss: 0.0041 - val_loss: 4.8080e-04
Epoch 5/100
51/51 [==============================] - 2s 34ms/step - loss: 0.0038 - val_loss: 4.2018e-04
Epoch 6/100
51/51 [==============================] - 2s 35ms/step - loss: 0.0032 - val_loss: 4.1674e-04
Epoch 7/100
51/51 [==============================] - 2s 37ms/step - loss: 0.0032 - val_loss: 8.7179e-04
Epoch 8/100
51/51 [==============================] - 2s 34ms/step - loss: 0.0032 - val_loss: 5.7302e-04
Epoch 9/100
51/51 [==============================] - 2s 35ms/step - loss: 0.0027 - val_loss: 4.4494e-04
Epoch 10/100
51/51 [==============================] - 2s 35ms/st

In [6]:
end_date = (datetime.today() - timedelta(days=0)).strftime('%Y-%m-%d')
start_date = (datetime.today() - timedelta(days=84)).strftime('%Y-%m-%d')
user_df = yf.download(tickers='EURUSD=X', start=start_date, end=end_date, interval='1d')
user_df = user_df.filter(['Close'])
user_data_value = user_df.values
user_data = scaler.transform(user_data_value)
forex_input = []
for i in range(60, len(user_data)):
    forex_input.append(user_data[i - 60:i, 0])
temp_input = list(forex_input)
temp_input = temp_input[0].tolist()
forex_input = np.array(forex_input)
forex_input = np.reshape(forex_input, (forex_input.shape[0], forex_input.shape[1], 1))



lstm_output = []
n_steps = 60
i = 0

while i < 10:

        if (len(temp_input) > n_steps):
            forex_input = np.array(temp_input[1:])
            forex_input = forex_input.reshape(1, -1)
            forex_input = forex_input.reshape((1, n_steps, 1))
            yhat = forexPredictor.predict(forex_input)
            temp_input.extend(yhat[0].tolist())
            temp_input = temp_input[1:]
            lstm_output.extend(yhat.tolist())
            i = i + 1
        else:
            forex_input = forex_input.reshape((1, n_steps, 1))
            yhat = forexPredictor.predict(forex_input)
            temp_input.extend(yhat[0].tolist())
            lstm_output.extend(yhat.tolist())
            i = i + 1

predictions = scaler.inverse_transform(lstm_output)


print(predictions)

[*********************100%***********************]  1 of 1 completed
[[1.22242786]
 [1.2206639 ]
 [1.21986435]
 [1.21934983]
 [1.21897187]
 [1.21866658]
 [1.21839332]
 [1.21812496]
 [1.21784472]
 [1.21754372]]


In [7]:
temp_input

[0.3535326394388507,
 0.3537242281231987,
 0.3874536011938434,
 0.3791119174222466,
 0.4030796955053093,
 0.42439721957342735,
 0.40584308449027295,
 0.4034345891315003,
 0.3925611736945034,
 0.3990846168765052,
 0.4109082313771757,
 0.41491576832113086,
 0.4161080627521967,
 0.4115819905289868,
 0.41690303794331385,
 0.41690303794331385,
 0.4101950770125904,
 0.426516143282746,
 0.43202137173457444,
 0.4308900220343821,
 0.44655584826984285,
 0.437150157042113,
 0.4756254747628206,
 0.4869871214692463,
 0.49575642849061774,
 0.48889526395465976,
 0.48657262995706274,
 0.4859513977556178,
 0.47686120494130435,
 0.49537325112192176,
 0.4922973941245239,
 0.49667329314348185,
 0.498199941816301,
 0.5109905909416601,
 0.5288221437460603,
 0.5156631325456984,
 0.5215320091154463,
 0.5073372742014555,
 0.5086800785479935,
 0.5067918020245772,
 0.5125468702018652,
 0.5177298660158711,
 0.5260638054294011,
 0.5393238299958516,
 0.5055345223279946,
 0.5254274212230428,
 0.5256819749055861,
 0.

In [8]:
forex_input= np.reshape(forex_input,(forex_input.shape[0]))
forex_input.shape

ValueError: cannot reshape array of size 60 into shape (1,)

In [ ]:

days=np.arange(1,11)

plt.plot(days,predictions)
print(predictions[2])